# The first lab results will be used to predict patient diagnoses for the X most common causes of death in ICU patients recorded in the MIMIC III database.

# MIMIC-III Critical Care Database

MIMIC-III (Medical Information Mart for Intensive Care III) is a large, freely-available database comprising deidentified health-related data associated with over forty thousand patients who stayed in critical care units of the Beth Israel Deaconess Medical Center between 2001 and 2012.

The database includes information such as demographics, vital sign measurements made at the bedside (~1 data point per hour), laboratory test results, procedures, medications, caregiver notes, imaging reports, and mortality (both in and out of hospital).

MIMIC supports a diverse range of analytic studies spanning epidemiology, clinical decision-rule improvement, and electronic tool development. It is notable for three factors:

it is freely available to researchers worldwide
it encompasses a diverse and very large population of ICU patients
it contains high temporal resolution data including lab results, electronic documentation, and bedside monitor trends and waveforms.

Citations: 
MIMIC-III, a freely accessible critical care database. Johnson AEW, Pollard TJ, Shen L, Lehman L, Feng M, Ghassemi M, Moody B, Szolovits P, Celi LA, and Mark RG. Scientific Data (2016). DOI: 10.1038/sdata.2016.35. Available at: http://www.nature.com/articles/sdata201635

Pollard, T. J. & Johnson, A. E. W. The MIMIC-III Clinical Database http://dx.doi.org/10.13026/C2XW26 (2016).

In [1]:
import sys
import pandas as pd
import datetime as datetime
import numpy as np
from dateutil.relativedelta import relativedelta
from sklearn.preprocessing import OneHotEncoder
import matplotlib
import matplotlib.pyplot as plt
#import psycopg2
from scipy.stats import ks_2samp
import scipy.stats as scats
import visuals as vs
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

plt.style.use('ggplot') 
%matplotlib inline

# IMPORTING DATA
The mimic III database was downloaded and reconstructed locally using posgresql. The database was managed graphically using Portico. 
The mimic III database was queried for information regarding ICD-9 codes and corresponding plain text diagnosis for patients who died while in the hospital. Data was then imported and analyzed to determine the X most common codes/diagnoses. 


In [23]:
diagnoses_codes = pd.DataFrame.from_csv('IDC9_DEADLY_DIAGNOSES.csv')
diagnoses_codes['icustay_id'] = diagnoses_codes.index
diagnoses_codes.set_index(np.arange(diagnoses_codes.shape[0]), inplace = True)

diagnoses_codes.head()

,diagnosis,icd9_code,short_title,icustay_id
0,DYSPNEA,48241,Meth sus pneum d/t Staph,280836
1,DYSPNEA,4280,CHF NOS,280836
2,DYSPNEA,51881,Acute respiratry failure,280836
3,DYSPNEA,5845,"Ac kidny fail, tubr necr",280836
4,DYSPNEA,4275,Cardiac arrest,280836


In [5]:
diagnoses_codes.shape

(94763, 3)

In [4]:
# data was grouped by the IDC-9 code and the plain text diagnoses. It appears that multiple codes may correspond 
# to the same plain text diagnoses so the same will be done for just ICD-9 codes and just plain text to verify. 
diagnoses_counts = diagnoses_codes.groupby(['icd9_code', 'short_title']).count()
diagnoses_counts.shape

(3254, 1)

In [17]:
diagnoses_counts.sort(['diagnosis'], ascending = False).head()

/Users/robertbeetel/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,,diagnosis
icd9_code,short_title,
51881,Acute respiratry failure,2668
4019,Hypertension NOS,2260
42731,Atrial fibrillation,2237
4280,CHF NOS,2195
5849,Acute kidney failure NOS,2132


In [30]:
# data was grouped by the IDC-9 code and the plain text diagnoses. It appears that multiple codes may correspond 
# to the same plain text diagnoses so the same will be done for just ICD-9 codes and just plain text to verify. 
diagnoses_codes_unique = diagnoses_codes.drop_duplicates('icustay_id')
diagnoses_counts3 = diagnoses_codes_unique.groupby(['diagnosis']).count()

display(diagnoses_counts3.shape)
diagnoses_counts3.head()


(2337, 3)

,icd9_code,short_title,icustay_id
diagnosis,,,
? SEROTONIN SYNDROME,1,1,1
(AML) ACUTE MYELOGENOUS LEUKEMIA,6,6,6
(AML) ACUTE MYELOGENOUS LEUKEMIA;HYPOTENSION,1,1,1
0681,1,1,1
12-AQV,1,1,1


In [31]:
diagnoses_counts3.sort(['icustay_id'], ascending = False).head(10)

/Users/robertbeetel/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,icd9_code,short_title,icustay_id
diagnosis,,,
SEPSIS,311,311,311
PNEUMONIA,289,289,289
INTRACRANIAL HEMORRHAGE,241,241,241
CONGESTIVE HEART FAILURE,145,145,145
ALTERED MENTAL STATUS,100,100,100
ABDOMINAL PAIN,96,96,96
HYPOTENSION,90,90,90
S/P FALL,84,84,84
CARDIAC ARREST,79,79,79


In [18]:
# data was grouped by the IDC-9 code and the plain text diagnoses. It appears that multiple codes may correspond 
# to the same plain text diagnoses so the same will be done for just ICD-9 codes and just plain text to verify. 
diagnoses_counts3 = diagnoses_codes.groupby(['diagnosis']).count()
display(diagnoses_counts3.shape)
diagnoses_counts3.head()


(2337, 2)

,icd9_code,short_title
diagnosis,,
? SEROTONIN SYNDROME,12,12
(AML) ACUTE MYELOGENOUS LEUKEMIA,106,106
(AML) ACUTE MYELOGENOUS LEUKEMIA;HYPOTENSION,25,25
0681,6,6
12-AQV,27,27


In [20]:
diagnoses_counts3.sort(['icd9_code'], ascending = False).head(10)

/Users/robertbeetel/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,icd9_code,short_title
diagnosis,,
SEPSIS,5145,5145
PNEUMONIA,4497,4497
CONGESTIVE HEART FAILURE,2379,2379
INTRACRANIAL HEMORRHAGE,2138,2138
ALTERED MENTAL STATUS,1706,1706
ABDOMINAL PAIN,1683,1683
HYPOTENSION,1536,1536
FEVER,1066,1066
S/P FALL,1045,1045


In [ ]:
diagnoses_list = diagnoses_codes.icd9_code.unique()
diagnoses_list2 = diagnoses_codes.diagnosis.unique()
display(len(diagnoses_list))
display(len(diagnoses_list2))
unique_codes = diagnoses_codes.drop_duplicates(['icd9_code', 'short_title'])
unique_codes.shape

diag_counts = diagnoses_codes.icd9_code.value_counts()
diag_counts[0:10]
diagnoses_list[0:10]

In [21]:
lab_results = pd.DataFrame.from_csv('DEADLY_DIAGNOSES.csv')
lab_results.groupby('diagnosis').count()

,hadm_id,icustay_id,mort_icu,mort_hosp,diagnosis,aniongap_1st,albumin_1st,bicarbonate_1st,bilirubin_1st,creatinine_1st,...,magnesium_1st,phosphate_1st,platelet_1st,potassium_1st,ptt_1st,inr_1st,pt_1st,sodium_1st,bun_1st,wbc_1st
subject_id,,,,,,,,,,,,,,,,,,,,,
21,109451,217847,0,0,CONGESTIVE HEART FAILURE,26.0,NaN,15.0,0.4,3.7,...,2.0,5.9,261.0,5.0,150.0,4.0,23.5,140.0,94.0,23.5
21,111970,216859,1,1,SEPSIS,19.0,2.1,23.0,0.3,4.2,...,1.5,4.4,275.0,3.8,31.7,1.7,18.1,137.0,49.0,30.5
37,188670,213503,0,0,CONGESTIVE HEART FAILURE,10.0,NaN,33.0,NaN,1.0,...,2.2,3.9,263.0,4.0,24.6,1.2,13.3,139.0,37.0,10.4
42,119203,210828,0,0,CONGESTIVE HEART FAILURE,14.0,3.6,25.0,0.3,0.7,...,1.9,2.8,228.0,4.3,25.7,1.6,15.4,139.0,18.0,7.9
68,170467,225771,0,0,PNEUMONIA,15.0,NaN,22.0,NaN,2.2,...,2.1,3.8,95.0,3.4,NaN,NaN,NaN,134.0,32.0,1.9


In [22]:
lab_results2 = lab_results.groupby(['diagnosis']).count()
display(lab_results2.shape)
lab_results2.head()

(4, 24)

,hadm_id,icustay_id,mort_icu,mort_hosp,aniongap_1st,albumin_1st,bicarbonate_1st,bilirubin_1st,creatinine_1st,chloride_1st,...,magnesium_1st,phosphate_1st,platelet_1st,potassium_1st,ptt_1st,inr_1st,pt_1st,sodium_1st,bun_1st,wbc_1st
diagnosis,,,,,,,,,,,,,,,,,,,,,
CONGESTIVE HEART FAILURE,1025,1025,1025,1025,1005,330,1011,406,1013,1013,...,986,941,1010,1013,897,899,899,1013,1012,1010
INTRACRANIAL HEMORRHAGE,754,754,754,754,717,243,718,254,735,732,...,684,681,732,734,712,714,715,732,735,732
PNEUMONIA,1662,1662,1662,1662,1646,671,1649,866,1651,1652,...,1604,1588,1648,1652,1387,1400,1400,1652,1651,1649
SEPSIS,1290,1290,1290,1290,1275,824,1275,1000,1277,1279,...,1257,1256,1275,1279,1185,1189,1189,1279,1277,1274


In [ ]:
labresults